<a href="https://colab.research.google.com/github/Tom-Jung/xml_fsa/blob/main/gemini_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import requests
import glob
import zipfile
import io
import os
import time

from io import BytesIO
import xml.etree.ElementTree as ET  # XML 파싱을 위해 추가
import logging  # 로깅을 위해 추가
from tqdm import tqdm

# DART API key and search period

In [11]:
crtfc_key = '6cc25146da26f08fc798e31bc27f02eaf67d6cd2'  # 기업통계팀 key로 변경
start_date = '20240101'
end_date = '20240530'   # 2025년 이후까지 늘려야?

# # 기업개황 개발가이드

In [12]:
# 공시대상회사의 고유번호(8자리)
corp_code = '01505760' # NAVER
url = f"https://opendart.fss.or.kr/api/company.json?crtfc_key={crtfc_key}&corp_code={corp_code}"
url

'https://opendart.fss.or.kr/api/company.json?crtfc_key=6cc25146da26f08fc798e31bc27f02eaf67d6cd2&corp_code=01505760'

In [13]:
def search_rcepts(corp_code, crtfc_key, start_date, end_date):
    url = f"https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={start_date}&end_de={end_date}&page_count=600"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        # 메시지 기반 예외 처리
        if data.get('status') != '000':
            print(f"[스킵] {corp_code}: {data.get('message')}")
            return pd.DataFrame()

        # list 키가 있을 경우만 데이터프레임 생성
        if 'list' in data:
            return pd.DataFrame(data['list'])
        else:
            print(f"[스킵] {corp_code}: list 없음")
            return pd.DataFrame()
    else:
        print(f"[에러] {corp_code}: status code {response.status_code}")
        return pd.DataFrame()

def download_and_extract_zip(url, extract_to='extracted_files'):
    """
    URL에서 ZIP 파일을 다운로드하고 지정된 디렉토리에 압축을 풉니다.

    Args:
        url (str): ZIP 파일을 다운로드할 URL
        extract_to (str): 파일을 압축 해제할 디렉토리 (기본값: 'extracted_files')
    """
    print(f"ZIP 파일 다운로드 중: {url}")

    # URL에서 파일 다운로드
    response = requests.get(url)

    # 응답 상태 확인
    if response.status_code == 200:
        # 압축 해제할 디렉토리 생성
        os.makedirs(extract_to, exist_ok=True)

        # 다운로드한 콘텐츠를 메모리 내 파일 객체로 변환
        zip_file = io.BytesIO(response.content)

        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            # ZIP 파일 내용 출력
            #print("ZIP 파일 내용:")
            file_list = zip_ref.namelist()
            for file in file_list:
                print(f"  {file}")

            # 모든 파일 압축 해제
            #print(f"압축 해제 중: {extract_to}")
            zip_ref.extractall(extract_to)

        #print(f"압축 해제 완료. 파일 개수: {len(file_list)}")
        return file_list
    else:
        print(f"다운로드 실패: 상태 코드 {response.status_code}")
        print(response.text)
        return None

In [15]:
r = search_rcepts('00156691', crtfc_key, start_date, end_date)
r['rcept_no'][0]
url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={r['rcept_no'][0]}"
files = download_and_extract_zip(url, 'test')

ZIP 파일 다운로드 중: https://opendart.fss.or.kr/api/document.xml?crtfc_key=6cc25146da26f08fc798e31bc27f02eaf67d6cd2&rcept_no=20240516001463
  20240516001463.xml


# DART API: 공시검색
- corp_rcept_no: 각 기업코드별 공시보고서 접수번호(rcept_no) 매칭

In [18]:
# API 1첨 돌렸을 때 (01652527) 금강에스티피 에서 한도 초과 발생(총 8243개 법인만 호출 완료)

start_time = time.time()  # 시작 시간

corp_rcept_no = pd.read_excel('corp_code.xlsx', dtype=str).set_index('corp_code')

reports = pd.DataFrame()

# 1000개 기업 공시 검색하는데 80초 소요
for i in corp_rcept_no.index[:1000]:
    r = search_rcepts(i, crtfc_key, start_date, end_date)
    reports = pd.concat([reports, r], axis=0)

end_time = time.time()  # 종료 시간

print(f"⏱ 실행 시간: {end_time - start_time:.2f}초")

⏱ 실행 시간: 637.37초


In [21]:
reports.to_excel('reports_FY2023.xlsx')

In [22]:
# reports.to_excel('reports_FY2023_1.xlsx', index=False)
# reports = pd.read_excel('reports_FY2023.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'reports_FY2023.xlsx'

In [23]:
reports.groupby(['corp_cls']).count()

,corp_code,corp_name,stock_code,report_nm,rcept_no,flr_nm,rcept_dt,rm
corp_cls,,,,,,,,
E,599,599,599,599,599,599,599,599
K,6660,6660,6660,6660,6660,6660,6660,6660
N,157,157,157,157,157,157,157,157
Y,7173,7173,7173,7173,7173,7173,7173,7173


In [24]:
# ER = reports.loc[reports['corp_cls'].eq('E')]
# ER['corp_code'].nunique() # 311개 기업의 공시가 있음

In [25]:
ER = reports
ER['corp_code'].nunique()

1000

In [26]:
# ER_audit = ER.loc[ER['report_nm'].str.contains('사업|감사') & ~ER['report_nm'].str.contains('연결감사|첨부정정') & ER['report_nm'].str.contains('2023.12')]
# # 2023.12 기준 사업 또는 감사 보고서가 있는 기업(298개)
# ER_audit['corp_code'].nunique()

In [27]:
ER_audit = ER.loc[
    ER['report_nm'].str.contains('사업|감사') &
    ~ER['report_nm'].str.contains('연결감사') &
    ~ER['report_nm'].str.contains('첨부정정') &
    ER['report_nm'].str.contains('2023.12', regex=False)
]
ER_audit['corp_code'].nunique()

999

In [28]:
ER_noaudit = ER.loc[~ER['corp_code'].isin(ER_audit['corp_code'].unique())]
# 2023.12 기준 사업 또는 감사 보고서가 없는 기업(13개)
ER_noaudit['corp_code'].nunique() # 2023.12 기준 사업 또는 감사 보고서가 없는 기업 1개

1

In [29]:
ER_audit['report_nm'].unique()

array(['사업보고서 (2023.12)', '감사보고서 (2023.12)', '[기재정정]사업보고서 (2023.12)',
       '[기재정정]감사보고서 (2023.12)', '[첨부추가]사업보고서 (2023.12)'], dtype=object)

In [30]:
ER_audit.to_csv('test.csv',encoding='utf-8-sig')

In [31]:
# ER_audit_latest.to_csv('test.csv',encoding='utf-8-sig')

In [32]:
ER_audit_one = ER_audit.sort_values('rcept_no', ascending=False).drop_duplicates(subset='corp_code', keep='first')

# DART API: 공시서류원본파일 다운로드

In [33]:
# extract_dir = "ER_audit"  # 압축 해제할 디렉토리 지정

# # 전체 1000개 기업중 기타(E)기업이고, 2023.12 기준 감사 또는 사업보고서가 있는 기업 공시보고서 다운로드하고 압축해제
# for i in ER_audit['rcept_no']:
#     url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={i}"
#     files = download_and_extract_zip(url, extract_dir)

In [34]:
def download_and_extract_zip(url, extract_to):
    response = requests.get(url)

    if response.status_code == 200:
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                z.extractall(extract_to)
                return z.namelist()
        except zipfile.BadZipFile:
            raise ValueError("Not a valid ZIP file (possibly an error response)")
    else:
        raise ValueError(f"Download failed with status code {response.status_code}")

extract_dir = "ER_audit"
os.makedirs(extract_dir, exist_ok=True)

for rcept_no in tqdm(ER_audit_one['rcept_no'], desc="Downloading and extracting reports"):
    url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={rcept_no}"
    try:
        files = download_and_extract_zip(url, extract_dir)
    except Exception as e:
        print(f"[SKIPPED] {rcept_no}: {e}")
        continue

In [36]:
ER_audit_one[ER_audit_one['corp_code'] == '01530030']

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,01530030,양명,,E,감사보고서 (2023.12),20240129000131,대주회계법인,20240129,


In [37]:
ER_audit_one

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
1,00595182,아톤,158430,K,[기재정정]사업보고서 (2023.12),20240516000621,아톤,20240516,연
2,00113243,대한제분,001130,Y,[기재정정]사업보고서 (2023.12),20240510001061,대한제분,20240510,연
3,00261054,EG,037370,K,[기재정정]사업보고서 (2023.12),20240510000991,EG,20240510,연
2,01142075,핑거,163730,K,[기재정정]사업보고서 (2023.12),20240430001125,핑거,20240430,연
10,00150633,지누스,013890,Y,[기재정정]사업보고서 (2023.12),20240425000282,지누스,20240425,연
...,...,...,...,...,...,...,...,...,...
0,01371224,대운개발,,E,감사보고서 (2023.12),20240207000766,우덕회계법인,20240207,
0,00203740,대아탱크터미널,,E,감사보고서 (2023.12),20240207000517,신승회계법인,20240207,
0,00206190,피에이치지,,E,감사보고서 (2023.12),20240206000482,삼화회계법인,20240206,
0,01530030,양명,,E,감사보고서 (2023.12),20240129000131,대주회계법인,20240129,


In [38]:
# import os
# import requests
# import zipfile
# import io
# from tqdm import tqdm

# def download_and_extract_zip(url, extract_to):
#     response = requests.get(url)
#     if response.status_code == 200:
#         try:
#             with zipfile.ZipFile(io.BytesIO(response.content)) as z:
#                 z.extractall(extract_to)
#                 return z.namelist()
#         except zipfile.BadZipFile:
#             raise ValueError("Not a valid ZIP file (possibly XML error message)")
#     else:
#         raise ValueError(f"HTTP error: {response.status_code}")

# # 압축 해제 디렉토리
# extract_dir = "ER_audit"
# os.makedirs(extract_dir, exist_ok=True)

# # 실패한 rcept_no 리스트 저장용
# skipped_list = []

# # 다운로드 및 압축 해제 루프
# for rcept_no in tqdm(ER_audit['rcept_no'], desc="Downloading and extracting reports"):
#     url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={rcept_no}"
#     try:
#         files = download_and_extract_zip(url, extract_dir)
#     except Exception as e:
#         print(f"[SKIPPED] {rcept_no}: {e}")
#         skipped_list.append(rcept_no)
#         continue

# # 필요시 로그 저장
# if skipped_list:
#     with open("skipped_rcept_no.txt", "w") as f:
#         for item in skipped_list:
#             f.write(f"{item}\n")

### 접수번호 - 기업코드 - 보고서명 - 접수일자 - xml파일경로 매핑

In [49]:
xml_files = pd.DataFrame({'path':glob.glob('ER_audit/*.xml')})
# xml_files['rcept_no'] = xml_files['path'].apply(lambda x: x.split('\\')[1][:14])
xml_files['rcept_no'] = xml_files['path'].apply(lambda x: os.path.basename(x)[:14])

In [50]:
xml_files

,path,rcept_no
0,ER_audit/20240307000835.xml,20240307000835
1,ER_audit/20240319000693.xml,20240319000693
2,ER_audit/20240318000674_00761.xml,20240318000674
3,ER_audit/20240318000694_00761.xml,20240318000694
4,ER_audit/20240319000487_00760.xml,20240319000487
...,...,...
2090,ER_audit/20240320001025.xml,20240320001025
2091,ER_audit/20240318000739.xml,20240318000739
2092,ER_audit/20240321000633_00760.xml,20240321000633
2093,ER_audit/20240315000795_00760.xml,20240315000795


In [52]:
xml_files.head()

,path,rcept_no
0,ER_audit/20240307000835.xml,20240307000835
1,ER_audit/20240319000693.xml,20240319000693
2,ER_audit/20240318000674_00761.xml,20240318000674
3,ER_audit/20240318000694_00761.xml,20240318000694
4,ER_audit/20240319000487_00760.xml,20240319000487


In [61]:
ER_audit_one['rcept_no'] = ER_audit_one['rcept_no'].astype('str')

In [62]:
ER_xml = pd.merge(xml_files, ER_audit_one, on='rcept_no')
tmp_xml_count = ER_xml.groupby('rcept_no')['path'].count().to_frame('xml_count').reset_index()
ER_xml = pd.merge(ER_xml, tmp_xml_count, on='rcept_no')
ER_xml.to_csv('ER_xml.csv', index=False,encoding='utf-8-sig')

In [63]:
ER_xml[ER_xml['corp_code'] == '01530030']

,path,rcept_no,corp_code,corp_name,stock_code,corp_cls,report_nm,flr_nm,rcept_dt,rm,xml_count
1956,ER_audit/20240129000131_00760.xml,20240129000131,01530030,양명,,E,감사보고서 (2023.12),대주회계법인,20240129,,1


In [64]:
tmp_xml_count = ER_xml.groupby('rcept_no')['path'].count().to_frame('xml_count').reset_index()

In [65]:
ER_xml

,path,rcept_no,corp_code,corp_name,stock_code,corp_cls,report_nm,flr_nm,rcept_dt,rm,xml_count
0,ER_audit/20240307000835.xml,20240307000835,00877059,삼성바이오로직스,207940,Y,사업보고서 (2023.12),삼성바이오로직스,20240307,연,3
1,ER_audit/20240319000693.xml,20240319000693,00121543,무학,033920,Y,사업보고서 (2023.12),무학,20240319,연,3
2,ER_audit/20240318000674_00761.xml,20240318000674,00578538,티피씨글로벌,130740,K,사업보고서 (2023.12),티피씨글로벌,20240318,연,3
3,ER_audit/20240318000694_00761.xml,20240318000694,00121932,미원상사,002840,Y,사업보고서 (2023.12),미원상사,20240318,연,3
4,ER_audit/20240319000487_00760.xml,20240319000487,00819073,인크로스,216050,K,사업보고서 (2023.12),인크로스,20240319,연,3
...,...,...,...,...,...,...,...,...,...,...,...
2087,ER_audit/20240320001025.xml,20240320001025,00455981,에스엔유프리시젼,080000,E,사업보고서 (2023.12),에스엔유프리시젼,20240320,연,3
2088,ER_audit/20240318000739.xml,20240318000739,00551920,네오팜,092730,K,사업보고서 (2023.12),네오팜,20240318,연,3
2089,ER_audit/20240321000633_00760.xml,20240321000633,00362122,나래나노텍,137080,K,사업보고서 (2023.12),나래나노텍,20240321,연,3
2090,ER_audit/20240315000795_00760.xml,20240315000795,00970453,파마리서치,214450,K,사업보고서 (2023.12),파마리서치,20240315,연,3
